In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

# Data Preprocessing

In this first part, we will process our two datasets :
- The first one is related to the **mRNA**, provided by the *Bälherlab (ULC)*.
- The second one is related to the **proteins**, provided by the *Sismanis-lab (EPFL)*.

We want to merge these 2 dataset into a coherent one, before further studies. We will proceed in 3 parts :
+ First we will describe our datasets;
+ Then we will process them independently;
+ Finally we will merge them.

## 0 - Loading data

In [2]:
# set data path
path_data_prot = "data/ratiowt.csv"
path_data_mRNA = "data/pat1_average_modified.txt"

# read data from files
raw_data_prot = pd.read_csv(path_data_prot, index_col=0)
raw_data_mRNA = pd.read_csv(path_data_mRNA, sep='\t', header=None)

## 1 - Datasets description

### a) Proteins data

In [3]:
# preview of the raw protein data
print(raw_data_prot.shape)
raw_data_prot.head()

(3281, 33)


,WT4_T00,WT4_T01,WT4_T02,WT4_T03,WT4_T04,WT4_T05,WT4_T06,WT4_T07,WT4_T08,WT4_T09,...,WT7_T01,WT7_T02,WT7_T03,WT7_T04,WT7_T05,WT7_T06,WT7_T07,WT7_T08,WT7_T09,WT7_T10
CON__P00761,13.150281,9.407338,24.657872,10.264727,15.709685,28.330217,15.511812,9.484966,22.430577,7.535227,...,23.298076,111.804298,70.269131,46.431722,71.108583,59.157596,67.168189,18.576311,10.721906,30.374825
CON__P04264,5.610728,8.141996,11.312985,7.446016,4.232088,8.690362,5.367110,6.333122,3.213471,4.681867,...,7.684033,7.000350,5.529750,3.083089,5.549082,8.059966,4.581272,2.864099,6.615507,2.820079
CON__P13645,3.163556,3.937628,7.504690,3.456978,6.605892,1.881963,0.889442,6.892749,0.561703,1.760192,...,7.459903,10.434928,7.730365,5.783356,16.064515,4.615527,2.237787,8.932559,6.195019,7.190623
CON__P15636,20.156007,6.966699,4.828119,13.614333,16.808700,9.455371,12.878798,21.559158,14.028197,7.663422,...,10.987683,7.503564,5.772006,17.601295,9.832842,24.968789,9.896091,17.334928,22.422027,24.981888
CON__P35527,6.656903,3.573726,3.286339,3.203588,1.679995,1.941107,2.872820,3.753331,4.186027,2.490412,...,2.876456,0.330950,0.935541,3.802426,3.579611,10.761714,4.365478,1.076577,5.539552,3.279979


- The dataset counts **2969 proteins**. <br/>
- For each protein, a relative concentration is measured for **11 time points** in **triplicate** (the 10 time points have been measured in 3 independent experiments).<br/>
- These 11 concentrations are measured for 0h, 1h, 2h, 3h, 4h, 5h, 6h, 7h, 8h, 9h, 10h.<br/>
- The 3 measures can differ (sometimes a lot in terms of order of magnitude), but the concentration's evolution seems to be the same (Averaged Pearson correlation coefficient = **0.8** between each measures A, B, C).

### b) mRNA data

In [4]:
# preview of the raw mRNA data
print(raw_data_mRNA.shape)
raw_data_mRNA.head()

(5113, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,aap1,AAP1,SPBC1652.02 SPBC16A3.20C,1.0,22.703531,0.960360,0.544460,0.896232,1.160251,1.252143,2.309153,2.310750,2.449327,2.706837,3.064384
1,abc1: C2D10.18,ABC1,SPBC2D10.18,1.0,1.043491,0.622283,0.663191,0.619409,0.648403,0.662236,0.655953,0.580761,0.606205,0.752322,0.798264
2,abc1: C9E9.12c,ABC1,SPAC9E9.12C,1.0,2.112546,0.906822,0.565746,0.535220,1.224398,10.170195,7.380023,4.455168,2.804587,1.657471,1.310630
3,abp1,CBP1 ABP1,SPBC1105.04C,1.0,1.351277,0.884985,0.705397,0.401055,0.202862,0.175360,0.829920,0.959749,1.121036,1.122949,1.103435
4,abp2,ABP2,SPBC1861.02,1.0,0.816301,1.969788,1.736116,1.057102,0.494822,0.372226,0.533438,0.851486,0.989702,1.115665,1.159631


- The data from the Bählerlab (ULC) is a table of measures for **5121 mRNAs**.<br/>
- For each mRNA a relative concentration is measured for **11 time points** (12 with t0, in which we are not so interested in). These relative concentrations are an **average** of 4 independent experiments.<br/>
- These 11 concentrations are measured for 0h, 1h, 2h, 3h, 4h, 5h, 6h, 7h, 8h, 10h, 12h<br/>
- As in the protein's dataset, most values are located between 1 and 2, but some values can be about several hundreds.<br/>
- The first 3 columns are respectively : the primer name, the common name and the systematic name of the mRNA. Only the systematic name corresponds to the name given in our first dataset.

For both datasets, **t0** is used as a ratio (thus, always equal to 1). In the proteins dataset, it is the time point at 0h (vegetative state), and in the mRNA dataset, it expresses another ratio, independent from time.

## 2 - Datasets preprocessing

### a) Proteins data

Our first preprocessing consists in averaging the triplicates at each time point. <br/>
Indead, it would be more simple to compute corralation between mRNA et proteins concentrations if we have a single value at each time point.

In [5]:
# split protein data into 3 sets
data_prot_A = pd.DataFrame(raw_data_prot.ix[:,0:11].values)
data_prot_B = pd.DataFrame(raw_data_prot.ix[:,11:22].values)
data_prot_C = pd.DataFrame(raw_data_prot.ix[:,22:33].values)

# compute average of the 3 data sets
data_prot = (data_prot_A + data_prot_B + data_prot_C).copy()/3.0

In order to get measures at the same time points in both datasets, we remove the 9th measure (= 9h, absent from the mRNA dataset).

In [6]:
data_prot = data_prot.drop(9, axis=1)

Finally, we choose to use the protein's name as index, and name the features as the time of the measurement.

In [7]:
# set columns and rows indexes
data_prot.columns = ['h'+str(k) for k in range(0,11) if k != 9]
data_prot.index = raw_data_prot.index

# show begining of dataframe
print(data_prot.shape)
data_prot.head()

(3281, 10)


,h0,h1,h2,h3,h4,h5,h6,h7,h8,h10
CON__P00761,62.703297,14.396267,57.974591,34.796653,24.998953,38.255215,37.618279,43.786914,42.088550,29.969380
CON__P04264,6.844208,7.315701,9.009564,7.077574,3.472497,6.154198,6.188080,5.591905,4.698724,6.585900
CON__P13645,5.565627,7.004613,9.074602,3.964237,5.710277,7.828472,3.950720,4.175028,4.711544,5.982843
CON__P15636,13.634706,9.466882,7.693673,9.907490,15.262446,11.852809,16.447941,14.847343,13.920279,16.075755
CON__P35527,6.987298,4.176773,3.488868,2.505684,4.240135,2.966327,5.963164,3.154968,2.449152,4.038704


### b) mRNA data

First of all, we remove the first 2 columns (the primer name and the common name) in which we are not interested.

In [8]:
# remove first 2 columns
data_mRNA = raw_data_mRNA.drop([0,1], axis=1).copy()

In order to get measures at the same time points in both datasets, we also remove t0 (which is meaningless for us) and t11 (=12h, which is missing in the proteins dataset).

In [9]:
data_mRNA = data_mRNA.drop([3,14], axis=1)

The remaining dataset counts 2434 missing values, involving 327 NaN entries. We chose to drop them :

In [10]:
data_mRNA.dropna(how='any', inplace=True)

Finally, as for the protein's dataset, we use the name as index and timing as columns.<br/>
Notice that some names are duplicates.

In [11]:
# set columns indexes
data_mRNA.set_index([2], inplace=True) # Some names are duplicates
data_mRNA.index.name = None
data_mRNA.columns = ['h'+str(k) for k in range(0,11) if k != 9]

print(data_mRNA.shape)
data_mRNA.head()

(4786, 10)


,h0,h1,h2,h3,h4,h5,h6,h7,h8,h10
SPBC1652.02 SPBC16A3.20C,22.703531,0.960360,0.544460,0.896232,1.160251,1.252143,2.309153,2.310750,2.449327,2.706837
SPBC2D10.18,1.043491,0.622283,0.663191,0.619409,0.648403,0.662236,0.655953,0.580761,0.606205,0.752322
SPAC9E9.12C,2.112546,0.906822,0.565746,0.535220,1.224398,10.170195,7.380023,4.455168,2.804587,1.657471
SPBC1105.04C,1.351277,0.884985,0.705397,0.401055,0.202862,0.175360,0.829920,0.959749,1.121036,1.122949
SPBC1861.02,0.816301,1.969788,1.736116,1.057102,0.494822,0.372226,0.533438,0.851486,0.989702,1.115665


## 3 - Merging the mRNA and protein data sets

We've got 2969 entries in our proteins dataset and 4786 entries in the mRNA one.<br/>
We'll first reduce the 2 datasets to their common entries, and then join the 2 tables.

### a) Filtering mRNA data

As mRNA names can be composed of several names, we extract the mRNA entries which name contains the one of our proteins.

In [12]:
# create the pattern and the filter
pattern = '|'.join(data_prot.index.values)
filter_ = data_mRNA.index.str.contains(pattern, case=False, na=False)

# extract mRNA data with corresponding protein name
data_mRNA = data_mRNA[filter_]

# show preview
print(data_mRNA.shape)
data_mRNA.head()

(3132, 10)


,h0,h1,h2,h3,h4,h5,h6,h7,h8,h10
SPBC2D10.18,1.043491,0.622283,0.663191,0.619409,0.648403,0.662236,0.655953,0.580761,0.606205,0.752322
SPBC1105.04C,1.351277,0.884985,0.705397,0.401055,0.202862,0.175360,0.829920,0.959749,1.121036,1.122949
SPBC32H8.12C,0.621846,0.325697,0.390388,0.419048,0.435039,0.539601,0.403898,0.337072,0.281851,0.265122
SPAC630.03,0.945575,0.576106,0.698916,0.638058,0.423130,0.352959,0.746192,0.829190,0.843786,0.889201
SPBC106.04,0.871751,0.869876,1.023361,0.727945,0.261486,0.143920,0.437443,0.805628,0.984534,0.974699


We've lost 95 entries, and still have 56 **duplicates**, representing 28 proteins.

In [13]:
duplicated_data_mRNA = data_mRNA.loc[data_mRNA[data_mRNA.index.duplicated()].index.sort_values()]
print(duplicated_data_mRNA.shape)
duplicated_data_mRNA.head()

(60, 10)


,h0,h1,h2,h3,h4,h5,h6,h7,h8,h10
SPAC13G7.02C,1.139014,5.825920,7.595293,19.737550,24.382723,27.602160,7.971517,6.919817,9.305873,12.082448
SPAC13G7.02C,0.480851,1.821550,2.200407,2.647362,3.204608,2.962069,0.816963,0.817025,0.790903,1.049258
SPAC144.03,0.597784,0.552436,0.423964,0.333494,0.287265,0.176862,0.230313,0.223476,0.261122,0.277014
SPAC144.03,1.247429,2.754731,2.002274,1.818740,1.673060,2.102656,2.313404,2.398567,3.019608,3.698596
SPAC16E8.15,0.300177,1.893358,0.955042,0.580376,0.673911,1.094835,0.801017,0.614706,0.535658,0.551441


To keep things simple, we chose to add them. <br/>
(As the duplicates are coding for the same protein, the sum of their concentration should be correlated with the proteins production)

In [14]:
duplicated_data_mRNA['name'] = duplicated_data_mRNA.index.values
duplicated_data_mRNA_sum = duplicated_data_mRNA.groupby(['name']).sum()
duplicated_data_mRNA_sum.index.name = None

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Finally, we replace them in our dataset :

In [15]:
data_mRNA_without_duplicates = data_mRNA.drop(duplicated_data_mRNA_sum.index)
data_mRNA = pd.concat([data_mRNA_without_duplicates, duplicated_data_mRNA_sum])

### b) Ending with mRNA duplicates and Filtering proteins data

Now, we have more proteins (2969) than we have mRNA (2858). Before extracting the common entries from the protein dataset, let's check if we have remaining duplicates :

In [16]:
duplicates = np.array([], dtype=object)

for name in data_prot.index.values:
    filter_ = data_mRNA.index.str.contains(name, case=False, na=False)
    
    if len(data_mRNA[filter_].index)>1:
        print(data_mRNA[filter_].index, name)
        duplicates = np.append(duplicates, data_mRNA[filter_].index)

Index(['SPAC145.04 SPAC20H4.10', 'SPAC20H4.10'], dtype='object') SPAC20H4.10
Index(['SPAC2E1P5.01C SPAPB1E7.13C', 'SPAPB1E7.13 SPAC2E1P5.01C'], dtype='object') SPAC2E1P5.01c


We still have one case of protein potentially produced by 2 distinct mRNA. Let's aggregate it as we did previously :

In [17]:
last_duplicate = data_mRNA.loc[duplicates]
last_duplicate_sum = last_duplicate.sum()
last_duplicate_sum.name = 'SPAC20H4.10'

data_mRNA = data_mRNA.drop(duplicates)
data_mRNA = data_mRNA.append(last_duplicate_sum)

Now, we can extract the common entries (the ARNm positions corresponding to proteins names) :

In [18]:
# get list of indexes and names to sort
idxname = np.array([], dtype=int)
listname = np.array([], dtype=object)

for name in data_prot.index.values:
    filter_ = data_mRNA.index.str.contains(name, case=False, na=False)
    
    if len(data_mRNA[filter_].index)>0:
        idxname = np.append(idxname, data_mRNA.index.get_loc(data_mRNA[filter_].index[0])) # mRNA numeric positions
        listname = np.append(listname, name) # Protein index

Let's filter the proteins dataset :

In [19]:
# removing non existant keys in protein data
data_prot = data_prot.loc[listname]
print(data_prot.shape)
data_prot.head()

(3099, 10)


,h0,h1,h2,h3,h4,h5,h6,h7,h8,h10
SPAC1002.01,0.559502,0.797365,0.714320,0.908648,0.979913,1.149922,1.088235,1.056883,1.176179,1.217080
SPAC1002.02,0.294818,0.409764,0.965734,1.713490,2.126497,1.714607,1.103619,0.844587,0.630046,0.423492
SPAC1002.03c,1.199934,1.098280,0.951916,0.900738,0.901141,0.933986,0.973173,1.017834,1.054168,0.996403
SPAC1002.04c,1.228882,1.183849,1.146613,1.283379,1.497678,1.406864,1.090262,0.863794,0.894849,0.997622
SPAC1002.07c,0.626058,0.687943,0.815417,1.015540,1.293387,1.316786,1.299641,1.251533,1.197197,1.105592


We've lost 124 entries, and now we've got the same number of entries in both datasets (2845).

### c) Joining the 2 datasets

Before merging the 2 datasets, we have to sort the mRNA dataset in the same order than the protein one :

In [20]:
data_mRNA = data_mRNA.iloc[list(idxname)]
data_mRNA.set_index([listname.tolist()], inplace=True)
data_mRNA.head()

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h10
SPAC1002.01,0.684485,1.199186,1.424561,1.597397,1.195754,1.172882,0.933155,0.819659,0.751874,0.683857
SPAC1002.02,1.035300,2.905877,4.153496,4.783120,2.603015,1.647060,1.481230,1.616345,1.347246,1.122512
SPAC1002.03c,0.958027,0.676712,0.703215,0.747138,0.861940,1.257074,1.235414,0.950157,0.988842,1.161624
SPAC1002.04c,1.133477,1.140642,1.189899,1.622498,2.308321,3.020603,1.853997,1.714919,1.682955,1.530989
SPAC1002.07c,0.783431,1.362667,1.602399,1.836938,1.938045,1.969526,0.570085,0.593492,0.524142,0.437590


Now, we can join the 2 datasets :

In [21]:
#We add a hierarchical column name
data_mRNA = pd.concat({"mRNA": pd.DataFrame(data_mRNA)}, axis=1)
data_prot = pd.concat({"prot": pd.DataFrame(data_prot)}, axis=1)

#And join
data = data_prot.join(data_mRNA)
print('Size of joined data:', data.shape)
data.head()

Size of joined data: (3099, 20)


prot                                                    \
                    h0        h1        h2        h3        h4        h5   
SPAC1002.01   0.559502  0.797365  0.714320  0.908648  0.979913  1.149922   
SPAC1002.02   0.294818  0.409764  0.965734  1.713490  2.126497  1.714607   
SPAC1002.03c  1.199934  1.098280  0.951916  0.900738  0.901141  0.933986   
SPAC1002.04c  1.228882  1.183849  1.146613  1.283379  1.497678  1.406864   
SPAC1002.07c  0.626058  0.687943  0.815417  1.015540  1.293387  1.316786   

                                                          mRNA            \
                    h6        h7        h8       h10        h0        h1   
SPAC1002.01   1.088235  1.056883  1.176179  1.217080  0.684485  1.199186   
SPAC1002.02   1.103619  0.844587  0.630046  0.423492  1.035300  2.905877   
SPAC1002.03c  0.973173  1.017834  1.054168  0.996403  0.958027  0.676712   
SPAC1002.04c  1.090262  0.863794  0.894849  0.997622  1.133477  1.140642   
SPAC1002.07c  1.299641  1.251533  1.197197  1.105592  0.783431  1.362667   

                                                                          \
                    h2        h3        h4        h5        h6        h7   
SPAC1002.01   1.424561  1.597397  1.195754  1.172882  0.933155  0.819659   
SPAC1002.02   4.153496  4.783120  2.603015  1.647060  1.481230  1.616345   
SPAC1002.03c  0.703215  0.747138  0.861940  1.257074  1.235414  0.950157   
SPAC1002.04c  1.189899  1.622498  2.308321  3.020603  1.853997  1.714919   
SPAC1002.07c  1.602399  1.836938  1.938045  1.969526  0.570085  0.593492   

                                  
                    h8       h10  
SPAC1002.01   0.751874  0.683857  
SPAC1002.02   1.347246  1.122512  
SPAC1002.03c  0.988842  1.161624  
SPAC1002.04c  1.682955  1.530989  
SPAC1002.07c  0.524142  0.437590

## 4 - Data variants

### a) Data in log2

In [22]:
data_prot_log2 = np.log2(data_prot)
data_mRNA_log2 = np.log2(data_mRNA)
data_log2 = np.log2(data)

### b) Data in difference of log2

In [23]:
def create_diff_log2(df):
    new_df = df.copy()
    new_df = new_df.diff(axis=1)
    new_df = new_df.drop('h0', axis=1)
    new_df.columns = ['d'+str(k) for k in range(1,11) if k != 9]
    
    return new_df

In [24]:
data_prot_geom = create_diff_log2(data_prot_log2.prot)
data_mRNA_geom = create_diff_log2(data_mRNA_log2.mRNA)

#We add a hierarchical column name
data_mRNA_geom = pd.concat({"mRNA": pd.DataFrame(data_mRNA_geom)}, axis=1)
data_prot_geom = pd.concat({"prot": pd.DataFrame(data_prot_geom)}, axis=1)

#And join
data_geom = data_prot_geom.join(data_mRNA_geom)

### c) Data Standardization (log2 and z-score)

As we will need to cluster our data, it could be useful to standardize them.<br/>
First of all, we define a suitable standization function :

In [25]:
def standardize_by_row(df):
    mean_rows = df.mean(axis=1)
    std_rows = df.std(axis=1)
    
    df = df.sub(mean_rows, axis=0)
    df = df.div(std_rows, axis=0)

    return df

We standardize the 10 measures of each mRNA and proteins independently :

In [26]:
data_prot_norm = standardize_by_row(data_prot_log2)
data_mRNA_norm = standardize_by_row(data_mRNA_log2)

And finally join the standardized datasets :

In [27]:
data_norm = data_prot_norm.join(data_mRNA_norm)

### Saving datasets

In [28]:
# Unstandardized data
data_prot.to_csv('data/data_prot.csv')
data_mRNA.to_csv('data/data_mRNA.csv')
data.to_csv('data/data.csv')

# Standardized data
data_prot_norm.to_csv('data/data_prot_log2_zscore.csv')
data_mRNA_norm.to_csv('data/data_mRNA_log2_zscore.csv')
data_norm.to_csv('data/data_log2_zscore.csv')

# Log2 data
data_prot_log2.to_csv('data/data_prot_log2.csv')
data_mRNA_log2.to_csv('data/data_mRNA_log2.csv')
data_log2.to_csv('data/data_log2.csv')

# Diff Log2 data
data_prot_geom.to_csv('data/data_prot_diff_log2.csv')
data_mRNA_geom.to_csv('data/data_mRNA_diff_log2.csv')
data_geom.to_csv('data/data_diff_log2.csv')